In [ ]:
tf.config.list_logical_devices()

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import cv2
import numpy as np
import tensorflow as tf
from keras.models import load_model
#from tkinter import Tk, filedialog
from skimage import img_as_ubyte
from skimage.segmentation import clear_border
from skimage import measure
import matplotlib.pyplot as plt
from keras.utils import normalize
import shutil
import pandas as pd
import pyvips
import re
import statistics
from tqdm import tqdm
logger = tf.get_logger()
logger.setLevel('ERROR')
# Image sizes need to be divisible by 16
IMG_HEIGHT = 688
IMG_WIDTH  = 432
coordinates = (300, 300, 740, 1000)  # Specify the ROI coordinates (left, upper, right, lower) PW (300, 300, 740, 1000)

In [2]:
# Colab import
file_path = "documents/Scan070602/Scan070602/Scan070602_00000000_0000.png"

file_dir, file_name = os.path.split(file_path)

name_numberAngPos = file_name.split('_0')[0] + "_0"

# Determination of the number of images per angular position
numberAngPos = len([f for f in os.listdir(file_dir) if f.startswith(name_numberAngPos) and f.endswith("_0000.png")]) - 1

# Determination of the total number of projection images
numberProjIm = len([f for f in os.listdir(file_dir) if f.startswith(name_numberAngPos + "0000000_") and f.endswith(".png")]) - 1


# Define Image Data List with Keywords
# The entire image is considered the ROI
imageData = {
    'filename': file_name,
    'path': file_dir + "/",
    'angPos': numberAngPos,
    'projPerAng': numberProjIm,
}


#return imageData, ImageROI
print(imageData)


{'filename': 'Scan070602_00000000_0000.png', 'path': 'documents/Scan070602/Scan070602/', 'angPos': 904, 'projPerAng': 64}


In [3]:

def generate_image_filenames(max_num1, max_num2):
    for num1 in range(max_num1 + 1):
        for num2 in range(max_num2+ 1):
            filename = f"Scan070602_{num1:08d}_{num2:04d}.png"
            filenames.append(filename)
    return filenames



In [4]:
def load_batch(image_paths):
    images = []
    for image_path in image_paths:
        image = pyvips.Image.new_from_file(image_path, access='sequential')
        memory_buffer = image.write_to_memory()
        # Convert the memory buffer to a NumPy array
        height, width = image.height, image.width
        np_image = np.frombuffer(memory_buffer, dtype=np.uint16).reshape(height, width)
        images.append(np_image)
        isinstance(np_image, np.ndarray)
    return images

In [5]:
def preprocess_images(batch_images):
    preprocessed_images = []
    for image in batch_images:
        image = image[coordinates[1]:coordinates[3], coordinates[0]:coordinates[2]]
        # Resize the image using OpenCV
        resized_image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
        # Append to the list of preprocessed images
        preprocessed_images.append(resized_image)
    #maybe change to -1
    preprocessed_images = np.expand_dims(normalize(np.array(preprocessed_images), axis=1),3)
    print(np.shape(preprocessed_images))
    return preprocessed_images



In [6]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda

################################################################
def simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):
#Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if inputs are normalized beforehand
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

    #Expansive path
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()

    return model


In [9]:
IMG_CHANNELS = 1

def get_model():
    return simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
model = get_model()
model.load_weights('documents/dropletonly.hdf5')


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 688, 432, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_19 (Conv2D)             (None, 688, 432, 16  160         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 dropout_9 (Dropout)            (None, 688, 432, 16  0           ['conv2d_19[0][0]']              
                                )                                                           

In [8]:
def post_processing(predictions):
    # Post-processing
    processed_images = []
    for prediction in predictions:
        # Binarize
        categoryImg_water = (prediction >= 0.3).astype(np.uint8)

        # Calculate buffer size (e.g., min dimension divided by 10)
        #buffer_size = min(categoryImg_water.shape) // 10
        #print(buffer_size)

        # Despeckling and clearing border
        #categoryImg_water_desp = clear_border(categoryImg_water, buffer_size=buffer_size)
        categoryImg_water_desp = categoryImg_water
        # Display despeckled image

        # Morphological operations
        se = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20))
        categoryImg_water_morph = cv2.morphologyEx(categoryImg_water_desp.astype(np.uint8), cv2.MORPH_OPEN, se)
        categoryImg_water_morph = cv2.morphologyEx(categoryImg_water_morph, cv2.MORPH_CLOSE, se)


        # Append to processed_images list
        processed_images.append(categoryImg_water)

    return processed_images


In [14]:
def get_regionprops(processed_images, min_area=600):

    for processed_image in processed_images:
        # Label the connected components
        labeled_droplets, _ = measure.label(processed_image, return_num=True, connectivity=2)
        # Find region properties
        regions = measure.regionprops(labeled_droplets)

        # Filter regions based on area
        filtered_regions = [region for region in regions if region.area >= min_area]

        # Find the lowest droplet among filtered regions
        if filtered_regions:
            # Select the droplet with the maximum vertical position (row component of centroid)
            lowest_droplet = max(filtered_regions, key=lambda region: region.centroid[0])

            # Create a binary image with just the lowest droplet
            test_img = np.zeros_like(processed_image, dtype=np.uint8)
            for coord in lowest_droplet.coords:
                test_img[coord[0], coord[1]] = 1
            lowest_droplet_props = {
                'centroid': lowest_droplet.centroid,
                'area': lowest_droplet.area,
                'bbox': lowest_droplet.bbox,
                'filled_area': lowest_droplet.filled_area
            }

        else:
            # Set default values if no droplet meets the criteria
            lowest_droplet_props = {
                'centroid': (0, 0),
                'area': 0,
                'bbox': (0, 0, 0, 0),
                'filled_area': 0
            }
        props[len(props) + 1] = lowest_droplet_props


In [15]:
filenames = []
image_filenames = generate_image_filenames(imageData['angPos'], imageData['projPerAng'])
batch_size = 16

if 'props' in locals():
    del props
props = {}
n = len(image_filenames)
for i in tqdm(range(0, n, batch_size), total=n // batch_size, desc="Processing batches"):
    batch_filenames = image_filenames[i:i + batch_size]
    batch_image_paths = [os.path.join(imageData['path'], filename) for filename in batch_filenames]

    # Load the batch
    batch_images = load_batch(batch_image_paths)

    # Preprocess the batch
    preprocessed_batch_images = preprocess_images(batch_images)

    # Predict images
    predictions = model.predict_on_batch(preprocessed_batch_images)
    
    
    processed_images = post_processing(predictions)

    get_regionprops(processed_images)

Processing batches:   0%|          | 0/3676 [00:00<?, ?it/s]

(16, 688, 432, 1)


Processing batches:   0%|          | 1/3676 [00:01<1:40:13,  1.64s/it]

(16, 688, 432, 1)


Processing batches:   0%|          | 2/3676 [00:03<1:45:04,  1.72s/it]

...

Processing batches: 100%|█████████▉| 3675/3676 [1:48:03<00:01,  1.55s/it]

(16, 688, 432, 1)


Processing batches: 100%|██████████| 3676/3676 [1:48:05<00:00,  1.54s/it]

(9, 688, 432, 1)


Processing batches: 3677it [1:48:07,  1.76s/it]                          


In [ ]:
#TEST
df = pd.DataFrame(props)
print(df)

In [16]:
#Add filenames to dictionary
pattern = r"Scan\d+_(\d+)_(\d+)\.png"

new_props = {key: {**props[key], 'filename': filename, 'AngPos': int(re.search(pattern, filename).group(1)), 'ProjperAng': int(re.search(pattern, filename).group(2))} for key, filename in zip(props.keys(), filenames)}

#Calculate average
total = 0
num_entries = len(props)

for values in new_props.values():
    cent = values['centroid']
    total += cent[0]

# Calculate the average height
average = total / num_entries
print(average)

411.73251795055717


In [19]:
#Centroid
# Extract the centroids
cents = [cent[0] for values in new_props.values() for cent in [values['centroid']]]


# Calculate the lower and upper quantiles (10% & 90%)
q1 = statistics.quantiles(cents, n=4)[0]
q3 = statistics.quantiles(cents, n=4)[-1]

# Calculate the interquantile range (IQR(yes, that name is technicaly wrong))(90-10)
iqr = q3 - q1

# Calculate the lower and upper bounds for outliers
lower_bound = q1 - (0.8 * iqr)
upper_bound = q3 + (0.8 * iqr)

# Remove outliers from the centroids list
filtered_cents = [cent for cent in cents if lower_bound <= cent <= upper_bound]
#print(sorted(filtered_cents))

# Find the minimum and maximum centroid hieght from the filtered centroids list
minimum_cent = min(filtered_cents)
maximum_cent = max(filtered_cents)
interval = (maximum_cent - minimum_cent) / 5

#steps = [minimum_cent + interval * i for i in range(6)]
steps = [334.76927647214154, 360.967734344973, 390.1661922178045, 420.36465009063596, 450.5631079634675, 480.7615658362989]
print(steps)
# Find the closest fitting bounding box for each unique AngPos

step_filenames = []

# Specify the parent directory to create the step folders
parent_directory = imageData['path']

for i, step in enumerate(steps):
    ang_positions = set(entry['AngPos'] for entry in new_props.values())

    # Initialize a list to store filenames for the current step
    current_step_filenames = []

    # Create the step directory
    step_directory = os.path.join(parent_directory, f'Step{i+1}')
    os.makedirs(step_directory)

    for ang_position in sorted(ang_positions):
        # Select the entries with the current 'AngPos' value
        selected_entries = [entry for entry in new_props.values() if entry['AngPos'] == ang_position]

        # Find the closest fitting regionprop value for the current step and 'AngPos'
        closest_fitting_cent = None
        min_distance = float('inf')

        for entry in selected_entries:
            cent = entry['centroid'][0]  # Calculate the height
            distance = abs(cent - step)

            if distance < min_distance:
                closest_fitting_cent = entry
                min_distance = distance

        # Append the filename for the closest fitting regionprop value to the current step filenames list
        if closest_fitting_cent is not None:
            filename = closest_fitting_cent['filename']
            current_step_filenames.append(filename)

            # Move the file to the step directory
            source_path = os.path.join(imageData['path'], filename)
            destination_path = os.path.join(step_directory, filename)
            shutil.copy(source_path, destination_path)

    # Append the current step filenames to the overall list
    step_filenames.append(current_step_filenames)

# Print the step filenames
# for i, filenames in enumerate(step_filenames):
#     print(f"Step {i+1} filenames: {filenames}")




[340.76927647214154, 360.967734344973, 390.1661922178045, 420.36465009063596, 450.5631079634675, 480.7615658362989]


In [18]:
print(steps)


[334.76927647214154, 364.967734344973, 395.1661922178045, 425.36465009063596, 455.5631079634675, 485.7615658362989]
